# Pseudo operators

In [1]:
import sys
sys.path.insert(0, "..") # dalgebra is here
from dalgebra import *
from dalgebra.dpolynomial.pseudo_doperator import *

In [2]:
R.<u,v> = DifferentialPolynomialRing(QQ)
S = PseudoDOperatorRing(R, "D")

## Computing a recursion operator

Let $L_n$ be a generic differential operator in normal form of order $n$. We know that the elements of the almost commuting basis $P_m$ satisfy
$$[L_n, P_m] = H_{m,0} + \ldots + H_{m,n-2}\partial^{n-2},$$
and the differential polynomials $H_{m,i}$ define the integrable hierarchy in some fashion. Let us write $H_m = (H_{m,0}, \ldots, H_{m,n-2})^T$.

The recursion operator is a matrix $\mathcal{R}$ of dimension $(n-1)\times(n-1)$ of pseudo-differential operators such that, for all $m \in \mathbb{N}$:
$$\mathcal{R} H_m = H_{m+n}.$$

In order to look for this recursion operator, we can observe that the polynomials $H_{m,i}$ are homogeneous of different weights (namely, they are homogeneous of order $n+m-i$). Hence, we can assume the coefficients of $\mathcal{R}$ must have appropriate homogeneous conditions. More precisely, the entries $R_{i,j}$ multiplies the coefficient $H_{m,j}$ and is used to create the coefficient $H_{m+n,i}$. Hence, it must be homogeneous of weight
$$w(R_{i,j}) = (m+n-i) - (m-j) = n+j-i.$$

Moreover, following the work [here](https://arxiv.org/abs/2406.19919), the authors argue that the appropriate shape of the recursion operator must be quasi-local, i.e., the pseudo-differential operators in the recursion matrix must be of the form
$$R_{i,j} = R_{i,j}^+ + u\partial^{-1}v,$$
where the elements $u,v$ are, respectively, a symmetry and cosymmetry of the differential equation associated.

Since the weights and homogeneous conditions hold in the ring of pseudo-differential operators, then we can set up a guess for these elements, since $w(u \partial^{-1} v) = w(u)+w(v)-1$.

### Necessary conditions on the pseudo part

Besides the homogeneous condition, we need that equation $(1)$ holds for **all** values of $m \in \mathbb{N}$. This can help us since we know the values for $H_1$ for all values of $n$:
$$H_1 = (u_n',\ldots,u_{2}')^T.$$
Since $\mathcal{R}H_1 = H_{n+1}$, it means that all operators in $\mathcal{R}$ must be applicable to these functions. More precisely, $R_{i,j}$ is applied to $H_{1,j}$, so the pseudo-part of $R_{i,j}$ must be compatible with $u_{n-j}'$.

Let us look to an example for $n=5$. 

In [3]:
R.<u_2,u_3,u_4,u_5,z> = DifferentialPolynomialRing(QQ)
S = PseudoDOperatorRing(R, "D")

In [4]:
w = R.weight_func([2,3,4,5,0], [1])

If we consider the element $R_{i,0}$, we know it has weight $5+0-i$ for $i=0,...,3$. So, namely:
$$w(R_{0,0}) = 5, \quad w(R_{1,0}) = 4, \quad w(R_{2,0}) = 3, \quad w(R_{3,0}) = 2.$$
This means that the pseudo part of $R_{3,0}$ must be a sum of the form $u\partial^{-1} v$ where $v$ has weight $0, 1, 2$ or $3$.

In [5]:
pseudo_for_R_3_0 = [(S.Di * el) for el in sum((w.homogeneous_monomials(i) for i in range(4)), tuple())]

In [6]:
valid = []
for el in pseudo_for_R_3_0:
    try:
        el(u_5[1])
        valid.append(el)
    except Exception as e:
        print(f"{el} was not valid: {e}")
valid

D^(-1)*(u_2_0) was not valid: [inverse_derivation] Element u_2_0*u_5_1 not integrable -> non-zero non-integrable part in decomposition: -u_2_1*u_5_0
D^(-1)*(u_2_1) was not valid: [inverse_derivation] Element u_2_1*u_5_1 not integrable -> non-zero non-integrable part in decomposition: -u_2_2*u_5_0
D^(-1)*(u_3_0) was not valid: [inverse_derivation] Element u_3_0*u_5_1 not integrable -> non-zero non-integrable part in decomposition: -u_3_1*u_5_0


[D^(-1)]

So we can conclude that the pseudo-part of $R_{0,3}$ can **only** be the simple $\partial^{-1}$.

We can now generalize this computation to check the pseudo valid parts for all the matrix.

In [11]:
def compute_integrability_conditions(element, *gens, c_name="a"):
    if len(gens) == 0:
        return False
    
    new_constants = [f"a_{i}" for i in range(len(gens))]
    R = element.parent()
    R = R.add_constants(*new_constants)
    constants = [R(R.base()(c)) for c in new_constants]
    element = R(element)
    gens = [R(el) for el in gens]
    generic_part = sum(c*g for (c,g) in zip(constants, gens))
    generic_element = element * generic_part

    solutions = integrability_conditions(generic_element, R)
    if len(solutions) == 0:
        return [generic_part, []]
    else:
        return [(sol.eval(generic_part), sol) for sol in solutions]

def integrability_conditions(element, parent):
    A, B = parent.integral_decomposition(element)

    if B == 0:
        return []
    else:
        conds = B.coefficients()
        conds = [conds[0].parent().wrapped(el) for el in conds]
        return analyze_ideal(ideal(conds), {}, [])
        

In [6]:
from dalgebra.commutators.almost_commuting import hierarchy
from dalgebra.commutators.ideals import analyze_ideal
H = [hierarchy(5,i) for i in range(1,5)]

In [7]:
H[0]

(-u_5_1, -u_4_1, -u_3_1, -u_2_1)

In [12]:
columns = [{i : compute_integrability_conditions(H[0][j], *w.homogeneous_monomials(i+1)) for i in range(1,5+j+1)} for j in range(4)]

In [13]:
columns[0]

{1: [(0, Solution Branch [a_0=0].)],
 2: [(0, Solution Branch [a_1=0,a_0=0].)],
 3: [(0, Solution Branch [a_2=0,a_0=0,a_1=0,a_3=0].)],
 4: [(a_5*u_5_0,
   Solution Branch [a_1=0,a_2=0,a_4=0,a_3=0,a_0=0] and a_5 as free variables.)],
 5: [(0,
   Solution Branch [a_0=0,a_7=0,a_1=0,a_8=0,a_5=0,a_9=0,a_6=0,a_4=0,a_10=0,a_3=0,a_2=0].)]}

In [10]:
Matrix([[5+j-i for j in range(4)] for i in range(4)])

[5 6 7 8]
[4 5 6 7]
[3 4 5 6]
[2 3 4 5]

From the previous two cells, we conclude that the pseudo-differential part of the first column of $\mathcal{R}$ has to be
* $R_{0,0} \longrightarrow \alpha_1 \partial^{-1} (u_5)$, where $w(\alpha_1) = 1$
* $R_{1,0} \longrightarrow \alpha_0 \partial^{-1} (u_5)$, where $w(\alpha_0) = 0$
* $R_{2,0} \longrightarrow$ no pseudo-part
* $R_{3,0} \longrightarrow$ no pseudo-part

For the rest of components of the matrix, we need to keep analyzing the object `columns`

In [14]:
columns[1]

{1: [(0, Solution Branch [a_0=0].)],
 2: [(0, Solution Branch [a_1=0,a_0=0].)],
 3: [(a_3*u_4_0,
   Solution Branch [a_2=0,a_0=0,a_1=0] and a_3 as free variables.)],
 4: [(0, Solution Branch [a_2=0,a_4=0,a_0=0,a_5=0,a_1=0,a_3=0].)],
 5: [(a_4*u_4_2,
   Solution Branch [a_1=0,a_6=0,a_9=0,a_5=0,a_10=0,a_7=0,a_3=0,a_0=0,a_8=0,a_2=0] and a_4 as free variables.)],
 6: [(0,
   Solution Branch [a_12=0,a_6=0,a_7=0,a_1=0,a_3=0,a_9=0,a_15=0,a_11=0,a_8=0,a_2=0,a_13=0,a_0=0,a_14=0,a_4=0,a_10=0,a_5=0].)]}

* $R_{0,1} \longrightarrow (\alpha_1 \partial^{-1} u_4'') + (\alpha_3 \partial^{-1} u_4)$, where $w(\alpha_1) = 1$ and $w(\alpha_3) = 3$
* $R_{1,1} \longrightarrow (\alpha_0 \partial^{-1} u_4'') + (\alpha_2 \partial^{-1} u_4)$, where $w(\alpha_0) = 0$ and $w(\alpha_2) = 2$
* $R_{2,1} \longrightarrow (\beta_1 \partial^{-1} u_4)$, where $w(\beta_1) = 1$
* $R_{3,1} \longrightarrow (\beta_0 \partial^{-1} u_4)$, where $w(\beta_0) = 0$

In [15]:
columns[2]

{1: [(0, Solution Branch [a_0=0].)],
 2: [(a_1*u_3_0, Solution Branch [a_0=0] and a_1 as free variables.)],
 3: [(0, Solution Branch [a_3=0,a_2=0,a_0=0,a_1=0].)],
 4: [(a_1*u_3_2,
   Solution Branch [a_4=0,a_5=0,a_2=0,a_0=0,a_3=0] and a_1 as free variables.)],
 5: [(a_10*u_3_0^2,
   Solution Branch [a_9=0,a_4=0,a_0=0,a_7=0,a_1=0,a_8=0,a_6=0,a_3=0,a_5=0,a_2=0] and a_10 as free variables.)],
 6: [((2*a_5)*u_2_0*u_3_2 + a_5*u_2_1*u_3_1 + a_1*u_3_4,
   Solution Branch [a_4=0,a_9=0,a_8=0,a_11=0,a_0=0,a_15=0,a_14=0,a_6=0,a_2=0,a_3=0,a_12=0,a_13=0,a_10=0,a_7=2*a_5] and a_1,a_5 as free variables.)],
 7: [((2*a_6)*u_2_1*u_3_2 + a_6*u_2_2*u_3_1 + (2*a_16)*u_3_0*u_3_2 + a_16*u_3_1^2,
   Solution Branch [a_15=0,a_1=0,a_5=0,a_9=0,a_11=0,a_8=0,a_28=0,a_4=0,a_21=0,a_24=0,a_0=0,a_10=0,a_23=0,a_25=0,a_18=0,a_12=0,a_26=0,a_19=0,a_27=0,a_22=0,a_20=0,a_3=0,a_13=0,a_14=0,a_2=0,a_7=2*a_6,a_17=2*a_16] and a_6,a_16 as free variables.)]}